In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
df = pd.read_csv("C:/Users/pkitt/Downloads/playground-series-s5e6/train.csv")

In [4]:
df.cat =df.select_dtypes(include=[object]) 

C:\Users\pkitt\AppData\Local\Temp\ipykernel_18232\1653857449.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.cat =df.select_dtypes(include=[object])


In [5]:
df.num=df.select_dtypes(include=[np.number])

C:\Users\pkitt\AppData\Local\Temp\ipykernel_18232\610118065.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.num=df.select_dtypes(include=[np.number])


In [6]:
df.cat.head()

,Soil Type,Crop Type,Fertilizer Name
0,Clayey,Sugarcane,28-28
1,Sandy,Millets,28-28
2,Sandy,Millets,17-17-17
3,Sandy,Barley,10-26-26
4,Red,Paddy,DAP


In [7]:
df.num.head()

,id,Temparature,Humidity,Moisture,Nitrogen,Potassium,Phosphorous
0,0,37,70,36,36,4,5
1,1,27,69,65,30,6,18
2,2,29,63,32,24,12,16
3,3,35,62,54,39,12,4
4,4,35,58,43,37,2,16


In [8]:
from sklearn.preprocessing import LabelEncoder

In [9]:
le = LabelEncoder()

In [10]:
df.cat["Soil Type"] = le.fit_transform(df.cat["Soil Type"])

In [11]:
df.cat["Crop Type"] = le.fit_transform(df.cat["Crop Type"])

In [12]:
df.cat["Fertilizer Name"] = le.fit_transform(df.cat["Fertilizer Name"])

In [13]:
df.cat.head()

,Soil Type,Crop Type,Fertilizer Name
0,1,8,4
1,4,4,4
2,4,4,2
3,4,0,0
4,3,6,5


In [14]:
df_train = pd.concat([df.num,df.cat],axis=1)

In [15]:
X = df_train[["Temparature","Humidity","Moisture","Crop Type","Soil Type"]]

In [16]:
y = df_train[["Fertilizer Name"]]

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:
import torch
import torch.nn as nn
from pytorch_tabnet.tab_network import TabNet

class FusionModel(nn.Module):
    def __init__(self, num_input_dim, cat_input_dim, tabnet_dim, transformer_dim, num_classes, cat_dims, cat_idxs, cat_emb_dim):
        super().__init__()
        self.embedding_proj = nn.Linear(num_input_dim, transformer_dim)
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=transformer_dim, nhead=4, batch_first=True),
            num_layers=2
        )
        self.tabnet = TabNet(
            input_dim=cat_input_dim,
            output_dim=tabnet_dim,
            n_d=tabnet_dim,
            n_a=tabnet_dim,
            n_steps=3,
            gamma=1.5,
            n_independent=2,
            n_shared=2,
            virtual_batch_size=128,
            momentum=0.02,
            mask_type="entmax",
            cat_dims=cat_dims,
            cat_idxs=cat_idxs,
            cat_emb_dim=cat_emb_dim
        )
        self.fusion = nn.Sequential(
            nn.Linear(transformer_dim + tabnet_dim, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x_num, x_cat):
        x_proj = self.embedding_proj(x_num)              # [B, 64]
        x_seq = x_proj.unsqueeze(1)                      # [B, 1, 64]
        x_trans = self.transformer(x_seq).squeeze(1)     # [B, 64]
        x_tabnet, _ = self.tabnet(x_cat)                 # [B, tabnet_dim]
        x_fused = torch.cat([x_trans, x_tabnet], dim=1)  # [B, combined_dim]
        return self.fusion(x_fused)

In [20]:
categorical_cols = ['Soil Type', 'Crop Type']  # or whatever your actual column names are

In [21]:
cat_idxs = [df.columns.get_loc(col) for col in categorical_cols]
cat_dims = [len(df[col].unique()) for col in categorical_cols]
cat_emb_dim = [min(50, (dim + 1) // 2) for dim in cat_dims]

In [22]:
from torch.utils.data import TensorDataset, DataLoader

dataset = TensorDataset(torch.tensor(X_num.values), torch.tensor(X_cat.values), torch.tensor(y))
loader = DataLoader(dataset, batch_size=64, shuffle=True)

model = FusionModel(
    num_input_dim=X_num.shape[1],
    cat_input_dim=X_cat.shape[1],
    tabnet_dim=32,
    transformer_dim=64,
    num_classes=num_classes,
    cat_dims=cat_dims,
    cat_idxs=cat_idxs,
    cat_emb_dim=cat_emb_dim
)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(5):
    model.train()
    for x_num_batch, x_cat_batch, y_batch in loader:
        optimizer.zero_grad()
        preds = model(x_num_batch.float(), x_cat_batch.int())
        loss = criterion(preds, y_batch)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

NameError: name 'X_num' is not defined

In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader, TensorDataset
from pytorch_tabnet.tab_network import TabNet
from pytorch_tabnet.tab_network import EmbeddingGenerator

# Save the original __init__ only once
_original_init = EmbeddingGenerator.__init__

def patched_init(self, input_dim, cat_dims, cat_idxs, cat_emb_dims, group_matrix=None):
    if group_matrix is None:
        # Create a dummy group matrix with shape [1, input_dim]
        import torch
        group_matrix = torch.zeros((1, input_dim))
    # Call the original unpatched __init__
    return _original_init(self, input_dim, cat_dims, cat_idxs, cat_emb_dims, group_matrix)

# Apply the patch
EmbeddingGenerator.__init__ = patched_init

# Load your data
df = pd.read_csv("C:/Users/pkitt/Downloads/playground-series-s5e6/train.csv")  # adjust path accordingly

categorical_cols = ['Crop Type', 'Soil Type']
numerical_cols = ['Temparature', 'Humidity', 'Moisture']
label_col = 'Fertilizer Name'

# Encode categoricals
for col in categorical_cols:
    df[col] = LabelEncoder().fit_transform(df[col])

# Encode label
label_encoder = LabelEncoder()
df[label_col] = label_encoder.fit_transform(df[label_col])
num_classes = len(label_encoder.classes_)

# Split features

X_cat = df[categorical_cols].astype(np.int32)
X_num = df[numerical_cols].astype(np.float32)
y = df[label_col].values

In [2]:
class FusionModel(nn.Module):
    def __init__(self, num_features, tabnet_input_dim, tabnet_dim, transformer_dim, num_classes, cat_idxs, cat_dims, cat_emb_dim):
        super().__init__()
        self.embedding_proj = nn.Linear(num_features, transformer_dim)
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=transformer_dim, nhead=4, batch_first=True),
            num_layers=2
        )
        self.tabnet = TabNet(
            input_dim=tabnet_input_dim, # 5 in your case
            output_dim=tabnet_dim,         # e.g. 32
            n_d=tabnet_dim,
            n_a=tabnet_dim,
            n_steps=3,
            gamma=1.5,
            n_independent=2,
            n_shared=2,
            virtual_batch_size=128,
            momentum=0.02,
            mask_type='entmax',
            cat_idxs=cat_idxs,
            cat_dims=cat_dims,
            cat_emb_dim=cat_emb_dim,# or use the min(50, (dim+1)//2) rule
            group_attention_matrix=None
        )
        self.fusion = nn.Sequential(
            nn.Linear(tabnet_dim + transformer_dim, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x_num, x_cat_full):
        x_proj = self.embedding_proj(x_num)
        x_seq = x_proj.unsqueeze(1)
        x_trans = self.transformer(x_seq).squeeze(1)
        x_tabnet, _ = self.tabnet(x_cat_full)
        x_fused = torch.cat([x_trans, x_tabnet], dim=1)
        return self.fusion(x_fused)

In [3]:
# Combine for TabNet: numerical + categorical
X_tab = pd.concat([X_num, X_cat], axis=1).astype(np.float32)

# Define categorical metadata
cat_idxs = list(range(X_num.shape[1], X_tab.shape[1]))
cat_dims = [len(df[col].unique()) for col in categorical_cols]
cat_emb_dim = [min(50, (dim + 1) // 2) for dim in cat_dims]

# Create dataset
dataset = TensorDataset(
    torch.tensor(X_num.values, dtype=torch.float32),
    torch.tensor(X_tab.values, dtype=torch.float32),
    torch.tensor(y, dtype=torch.long)  # ensure target dtype is correct
)
loader = DataLoader(dataset, batch_size=64, shuffle=True)

# Instantiate model, loss, optimizer
model = FusionModel(
    num_features=X_num.shape[1],
    tabnet_input_dim=X_tab.shape[1],
    tabnet_dim=32,
    transformer_dim=64,
    num_classes=num_classes,
    cat_idxs=cat_idxs,
    cat_dims=cat_dims,
    cat_emb_dim=cat_emb_dim
)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
# Training loop
for epoch in range(5):
    model.train()
    running_loss = 0.0
    for xb_num, xb_tab, yb in loader:
        optimizer.zero_grad()
        outputs = model(xb_num, xb_tab)
        loss = criterion(outputs, yb)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1} | Loss: {running_loss/len(loader):.4f}")

Epoch 1 | Loss: 1.9431
Epoch 2 | Loss: 1.9426
Epoch 3 | Loss: 1.9426
Epoch 4 | Loss: 1.9426
Epoch 5 | Loss: 1.9426
